In [3]:
import json

In [45]:
answer = json.load(open("data/gg2013answers.json", "r"))


In [46]:
for i in answer['award_data'].keys():
    print(i)

best screenplay - motion picture
best director - motion picture
best performance by an actress in a television series - comedy or musical
best foreign language film
best performance by an actor in a supporting role in a motion picture
best performance by an actress in a supporting role in a series, mini-series or motion picture made for television
best motion picture - comedy or musical
best performance by an actress in a motion picture - comedy or musical
best mini-series or motion picture made for television
best original score - motion picture
best performance by an actress in a television series - drama
best performance by an actress in a motion picture - drama
cecil b. demille award
best performance by an actor in a motion picture - comedy or musical
best motion picture - drama
best performance by an actor in a supporting role in a series, mini-series or motion picture made for television
best performance by an actress in a supporting role in a motion picture
best television serie

In [6]:
ans = dict()
for i in answer['award_data'].keys():
    tmp = i.split(' ')
    for j in tmp:
        if j in ans:
            ans[j] += 1
        else:
            ans[j] = 1
print(len(answer['award_data'].keys()))

26


In [8]:
ans = sorted(ans.items(), key=lambda x: x[1], reverse=True)
for i in ans:
    print(i)

('best', 25)
('in', 18)
('a', 18)
('motion', 17)
('picture', 17)
('-', 16)
('performance', 14)
('by', 14)
('an', 14)
('television', 11)
('or', 11)
('actress', 7)
('actor', 7)
('series', 6)
('comedy', 6)
('musical', 6)
('drama', 6)
('mini-series', 5)
('made', 5)
('for', 5)
('supporting', 4)
('role', 4)
('film', 2)
('series,', 2)
('original', 2)
('screenplay', 1)
('director', 1)
('foreign', 1)
('language', 1)
('score', 1)
('cecil', 1)
('b.', 1)
('demille', 1)
('award', 1)
('animated', 1)
('feature', 1)
('song', 1)


In [10]:
import pandas as pd

In [12]:
data = pd.read_csv("data/actors.tsv", sep='\t')

In [13]:
data

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
0,nm0000001,Fred Astaire,1899,1987,"soundtrack,actor,miscellaneous","tt0053137,tt0050419,tt0072308,tt0031983"
1,nm0000002,Lauren Bacall,1924,2014,"actress,soundtrack","tt0117057,tt0037382,tt0038355,tt0075213"
2,nm0000003,Brigitte Bardot,1934,\N,"actress,soundtrack,music_department","tt0049189,tt0054452,tt0056404,tt0057345"
3,nm0000004,John Belushi,1949,1982,"actor,soundtrack,writer","tt0077975,tt0080455,tt0072562,tt0078723"
4,nm0000005,Ingmar Bergman,1918,2007,"writer,director,actor","tt0069467,tt0050986,tt0050976,tt0083922"
...,...,...,...,...,...,...
12931040,nm9993714,Romeo del Rosario,\N,\N,"animation_department,art_department","tt14069590,tt11657662,tt2455546"
12931041,nm9993716,Essias Loberg,\N,\N,NaN,\N
12931042,nm9993717,Harikrishnan Rajan,\N,\N,cinematographer,tt8736744
12931043,nm9993718,Aayush Nair,\N,\N,cinematographer,tt8736744


In [16]:
type(data['deathYear'][2])

str

In [17]:
import re

In [35]:
match = re.search(r'\b(\S+)\s+wins\s+#goldenglobe\s+for\s+(\S+)\s+for\s+(\S+)', "xxx wins #goldenglobe for AWARD for role/movie")
if match:
    print(match.group(2))

AWARD


In [2]:
def look_forward(sent, ind, start=None, end=None, include=True, start_exclude=None, end_exclude=None, max_len=99):
    # speed up search by anchoring first to start
    if start is not None and not include and len(start) <= len(sent) - ind and sent[ind+1:ind+1+len(start)] == start:
        ind += len(start)
        start = None
    ngrams = []
    offset = None
    end_offset = 0
    if include:
        offset = 0
    elif start is not None and end is None:
        offset = len(start)
    elif end is not None and start is None:
        offset = len(end)
    elif start is None and end is None:
        offset = 0
    else:
        start_offset = len(start)
        end_offset = len(end)
        offset = start_offset
    for length in range(1, min(max_len + 1, len(sent) - ind)):
        if start_exclude is not None and sent[ind+1+offset] in start_exclude:
            break
        elif end_exclude is not None and sent[ind+length-end_offset] in end_exclude:
            break
        elif start is None and end is None:
            res = sent[ind+1: ind+length+1]
            if len(res) > 0:
                ngrams.append(res)
        elif start is not None and end is None and len(start) <= length and sent[ind+1: ind+len(start)+1] == start:
            res = sent[ind+1+offset: ind+length+1]
            if len(res) > 0:
                ngrams.append(res)
        elif end is not None and start is None and len(end) <= length and sent[ind+length+1-len(end):ind+length+1] == end:
            res = sent[ind+1: ind+length+1-offset]
            if len(res) > 0:
                ngrams.append(res)
        elif start is not None and end is not None and len(start) <= length and len(end) <= length and sent[ind+1: ind+len(start)+1] == start and sent[ind+length+1-len(end):ind+length+1] == end:
            res = sent[ind+1+start_offset: ind+length+1-end_offset]
            if len(res) > 0:
                ngrams.append(res)
                # greedy match end token
                break
    return ngrams


def look_backward(sent, ind, start=None, end=None, include=True, start_exclude=None, end_exclude=None, max_len=99):
    if end is not None and not include and len(end) <= len(sent) - ind and sent[ind-len(end):ind] == end:
        ind -= len(end)
        end = None
    ngrams = []
    offset = None
    start_offset = 0
    if include:
        offset = 0
    elif start is not None and end is None:
        offset = len(start)
    elif end is not None and start is None:
        offset = len(end)
    elif start is not None and end is not None:
        start_offset = len(start)
        end_offset = len(end)
        offset = end_offset
    else:
        offset = 0
    for length in range(1, min(max_len + 1, ind+1)):
        if end_exclude is not None and sent[ind - 1 - offset] in end_exclude:
            break
        elif start_exclude is not None and sent[ind - length + start_offset] in start_exclude:
            break
        elif start is None and end is None:
            res = sent[ind - length: ind]
            if len(res) > 0:
                ngrams.append(res)
        elif start is not None and end is None and len(start) <= length and sent[ind-length: ind-length+len(start)] == start:
            res = sent[ind - length + offset: ind]
            if len(res) > 0:
                ngrams.append(res)
        elif end is not None and start is None and len(end) <= length and sent[ind-len(end):ind] == end:
            res = sent[ind - length: ind - offset]
            if len(res) > 0:
                ngrams.append(res)
        elif start is not None and end is not None and len(start) <= length and len(end) <= length and sent[ind-length: ind-length+len(start)] == start and sent[ind-len(end):ind] == end:
            res = sent[ind - length + start_offset: ind - end_offset]
            if len(res) > 0:
                ngrams.append(res)
                # greedy match end token
                break
    return ngrams

In [4]:
data = json.load(open("data/gg2013.json", "r"))

In [23]:

ca_set_awards = []
sents = [x['text'].lower().split() for x in data]
for sent in sents:
    for i, word in enumerate(sent):
        if word == 'goes' and i+1 < len(sent) and sent[i+1] == 'to':
            ca_set_awards += look_backward(sent, i+2, end=['goes', 'to'], start_exclude=['at', 'the', 'golden', 'globes', 'globe', 'for'], end_exclude=['at', 'the', 'golden', 'globes', 'globe'], include=False)
        # xxx wins #goldenglobe for AWARD for role/movie
        elif word == 'wins' and i+1 < len(sent) and sent[i+1] == 'for':
            ca_set_awards += look_forward(sent, i-1, start=['wins', 'for'], end=['for'], end_exclude=['at', 'the', 'golden', 'globes', 'globe'], include=False)
        elif word == 'won' and i+1 < len(sent) and sent[i+1] == 'for':
            ca_set_awards += look_forward(sent, i-1, start=['won', 'for'], end=['for'], end_exclude=['at', 'the', 'golden', 'globes', 'globe'], include=False)
        elif word == 'won' and i+1 < len(sent) and sent[i+1] == 'for' and i > 0 and sent[i-1] == 'has':
            ca_set_awards += look_forward(sent, i-2, start=['has', 'won', 'for'], end=['for'], end_exclude=['at', 'the', 'golden', 'globes', 'globe'], include=False)
        # AWARD is awarded to xxx
        elif word == 'awarded' and 1 < i+1 < len(sent) and sent[i+1] == 'to' and sent[i-1] == 'is':
            ca_set_awards += look_backward(sent, i+2, end=['is', 'awarded', 'to'], include=False)
        # AWARD (#goldenglobe) awarded to xxx
        elif word == 'awarded' and i+1 < len(sent) and sent[i+1] == 'to':
            ca_set_awards += look_backward(sent, i+2, end=['awarded', 'to'], include=False, end_exclude=['is'])
        # winner for AWARD is xxx
        elif word == 'winner' and i+3 < len(sent) and sent[i+1] == 'for':
            ca_set_awards += look_forward(sent, i-1, start=['winner', 'for'], end=['is'], end_exclude=['at', 'the', 'golden', 'globes', 'globe'], include=False)
        # someone presents AWARD to xxx
        elif word == 'presents' and i+2<len(sent) and sent[i+2] == 'to':
            ca_set_awards += look_forward(sent, i-1, start=['presents'], end=['to'], include=False)
        # xxx wins/won/has won the #goldenglobe for AWARD
        elif word == 'won' and i+3 < len(sent) and i>0 and sent[i-1] == 'has' and sent[i + 1] == 'the' and sent[i + 2] == 'for':
            ca_set_awards += look_forward(sent, i-2, start=['has', 'won', 'the', 'for'], end=['for'], end_exclude=['at', 'the', 'golden', 'globes', 'globe'], include=False)
        elif word == 'won' and i+3 < len(sent) and sent[i+1] == 'the' and sent[i+2] == 'for':
            ca_set_awards += look_forward(sent, i-1, start=['won', 'the', 'for'], end=['for'], end_exclude=['at', 'the', 'golden', 'globes', 'globe'], include=False)
        elif word == 'wins' and i + 4 < len(sent) and sent[i + 1] == 'the' and sent[i + 2] == 'award' and sent[i + 3] == 'for':
            ca_set_awards += look_forward(sent, i - 1, start=['wins', 'the', 'award', 'for'], end=['for'], end_exclude=['at', 'the', 'golden', 'globes', 'globe'], include=False)
        elif word == 'wins' and i+3 < len(sent) and sent[i+1] == 'the' and sent[i+2] == 'for':
            ca_set_awards += look_forward(sent, i-1, start=['wins', 'the', 'for'], end=['for'], end_exclude=['at', 'the', 'golden', 'globes', 'globe'], include=False)
        elif word == 'win' and i + 3 < len(sent) and sent[i + 1] == 'the' and sent[i + 2] == 'for':
            ca_set_awards += look_forward(sent, i - 1, start=['win', 'the', 'for'], end=['for'], end_exclude=['at', 'the', 'golden', 'globes', 'globe'], include=False)
        # xxx wins/won/has won the golden globe for AWARD
        elif word == 'won' and i + 5 < len(sent) and i>0 and sent[i-1] == 'has' and sent[i + 1] == 'the':
            ca_set_awards += look_forward(sent, i-2, start=['has', 'won', 'the', 'golden', 'globe', 'for'], end=['for'], end_exclude=['at', 'the', 'golden', 'globes', 'globe'], include=False)
        elif word == 'won' and i + 5 < len(sent) and sent[i+1] == 'the' and sent[i+2] == 'for':
            ca_set_awards += look_forward(sent, i-1, start=['won', 'the', 'golden', 'globe', 'for'], end=['for'], end_exclude=['at', 'the', 'golden', 'globes', 'globe'], include=False)
        elif word == 'wins' and i + 5 < len(sent) and sent[i+1] == 'the' and sent[i+2] == 'for':
            ca_set_awards += look_forward(sent, i-1, start=['wins', 'the', 'golden', 'globe', 'for'], end=['for'], end_exclude=['at', 'the', 'golden', 'globes', 'globe'], include=False)
        # congrat/congrats to xxx for her/his golden globe win as AWARD
        elif word == 'win' and 4 < i < len(sent)-1 and sent[i + 1] == 'as' and sent[i - 1] == 'globe' and sent[i - 2] == 'golden':
            ca_set_awards += look_forward(sent, i-5, start=['for', 'his', 'golden', 'globe', 'win', 'as'], end=['for'], end_exclude=['at', 'the', 'golden', 'globes', 'globe'], include=False)
            ca_set_awards += look_forward(sent, i-5, start=['for', 'her', 'golden', 'globe', 'win', 'as'], end=['for'], end_exclude=['at', 'the', 'golden', 'globes', 'globe'], include=False)
        # xxx wins AWARD for role/movie
        elif word == 'wins':
            ca_set_awards += look_forward(sent, i-1, start=['wins'], end=['for'], end_exclude=['at', 'the', 'golden', 'globes', 'globe'], include=False)
        elif word == 'won' and i > 0 and sent[i-1] == 'has':
            ca_set_awards += look_forward(sent, i-2, start=['has', 'won'], end=['for'], end_exclude=['at', 'the', 'golden', 'globes', 'globe'], include=False)
        elif word == 'won':
            ca_set_awards += look_forward(sent, i-1, start=['won'], end=['for'], end_exclude=['at', 'the', 'golden', 'globes', 'globe'], include=False)
ca_set_awards


[['ever"'],
 ['icon', 'ever"'],
 ['twitter', 'icon', 'ever"'],
 ['"best', 'twitter', 'icon', 'ever"'],
 ['to', '"best', 'twitter', 'icon', 'ever"'],
 ['#goldenglobes'],
 ['zinger'],
 ['first', 'zinger'],
 ['camera'],
 ['best', 'supporting', 'actor', 'in', 'a', 'motion', 'picture'],
 ['best', 'supporting', 'actor', 'in', 'a', 'motion', 'picture'],
 ['best', 'supporting', 'actor', 'in', 'a', 'motion', 'picture'],
 ['best', 'supporting', 'actor'],
 ['picture'],
 ['motion', 'picture'],
 ['a', 'motion', 'picture'],
 ['in', 'a', 'motion', 'picture'],
 ['actor', 'in', 'a', 'motion', 'picture'],
 ['supporting', 'actor', 'in', 'a', 'motion', 'picture'],
 ['best', 'supporting', 'actor', 'in', 'a', 'motion', 'picture'],
 ['@tvguide:', 'best', 'supporting', 'actor', 'in', 'a', 'motion', 'picture'],
 ['rt',
  '@tvguide:',
  'best',
  'supporting',
  'actor',
  'in',
  'a',
  'motion',
  'picture'],
 ['best', 'supporting', 'actor!', "that's", 'one'],
 ['movie'],
 ['a', 'movie'],
 ['in', 'a', 'movie'

In [6]:
def unique_ngrams(ngrams_lst):
    dict = {}
    cnt = []
    for ngram in ngrams_lst:
        ngram_str = ' '.join(ngram)
        if ngram_str not in dict:
            dict[ngram_str] = 1
        else:
            dict[ngram_str] += 1
    for ngram_str in dict:
        cnt.append([ngram_str.split(), dict[ngram_str]])
    sorted_cnt = sorted(cnt, key=lambda x: x[1], reverse=True)
    return sorted_cnt

In [24]:
sorted_awards = unique_ngrams(ca_set_awards)[:400]
sorted_awards

[[['picture'], 209],
 [['motion', 'picture'], 206],
 [['drama'], 185],
 [['than', 'happy'], 163],
 [['tonight'], 126],
 [['picture', 'drama'], 126],
 [['motion', 'picture', 'drama'], 126],
 [['@theellenshow:', 'tonight'], 123],
 [['rt', '@theellenshow:', 'tonight'], 123],
 [['a', 'motion', 'picture'], 99],
 [['best', 'supporting', 'actress', 'in', 'a', 'motion', 'picture'], 94],
 [['best', 'motion', 'picture', 'drama'], 94],
 [['in', 'a', 'motion', 'picture'], 93],
 [['best', 'director'], 93],
 [['film'], 92],
 [['movie'], 89],
 [['best', 'original', 'song'], 85],
 [['best', 'actor'], 83],
 [['actress', 'in', 'a', 'motion', 'picture'], 82],
 [['supporting', 'actress', 'in', 'a', 'motion', 'picture'], 82],
 [['original', 'song'], 80],
 [['musical'], 75],
 [['or', 'musical'], 74],
 [['comedy', 'or', 'musical'], 74],
 [['best', 'actress'], 73],
 [['#goldenglobe'], 70],
 [['best',
   'actor',
   'in',
   'a',
   'motion',
   'picture,',
   'comedy',
   'or',
   'musical'],
  63],
 [['@cnns

In [25]:
def filter_award_sorted(lst):
    remove = []
    for i, award in enumerate(lst):
        if award[0][0] != 'best' or len(award[0]) <= 2:
            remove.append(i)
    remove = sorted(remove, reverse=True)
    for idx in remove:
        lst.remove(lst[idx])
    return lst

In [26]:
sorted_awards = filter_award_sorted(sorted_awards)
sorted_awards

[[['best', 'supporting', 'actress', 'in', 'a', 'motion', 'picture'], 94],
 [['best', 'motion', 'picture', 'drama'], 94],
 [['best', 'original', 'song'], 85],
 [['best',
   'actor',
   'in',
   'a',
   'motion',
   'picture,',
   'comedy',
   'or',
   'musical'],
  63],
 [['best', 'supporting', 'actress'], 57],
 [['best', 'director', 'motion', 'picture', '#goldenglobe'], 54],
 [['best', 'supporting', 'actor,', 'motion', 'picture'], 49],
 [['best', 'animated', 'feature', 'film'], 44],
 [['best', 'supporting', 'actor'], 43],
 [['best', 'actress', 'in', 'a', 'tv', 'comedy', 'or', 'musical'], 29],
 [['best', 'actor,', 'drama'], 26],
 [['best',
   'supporting',
   'actress',
   'tv',
   'series,',
   'miniseries,',
   'or',
   'tv',
   'movie'],
  22],
 [['best', 'original', 'score', '-', 'motion', 'picture'], 18],
 [['best', 'supporting', 'actor', 'in', 'a', 'motion', 'picture'], 17],
 [['best',
   'original',
   'song',
   'for',
   'skyfall,',
   'because',
   'it',
   'is',
   'not',
   

In [17]:
def remove_all_sublists(sorted_ca):
    removal = []
    for ca1 in sorted_ca:
        for ca2 in sorted_ca:
            if is_StrictSublist(ca2[0], ca1[0]):
                if ca1 not in removal:
                    removal.append(ca1)
    for to_remove in removal:
        sorted_ca.remove(to_remove)
    return [x[0] for x in sorted_ca]

In [21]:
def is_StrictSublist(l, s):
    sub_set = False
    if s == []:
        sub_set = True
    elif len(s) >= len(l):
        sub_set = False
    else:
        l_string = ' '.join(l)
        s_string = ' '.join(s)
        sub_set = s_string in l_string
    return sub_set

In [27]:
sorted_awards = remove_all_sublists(sorted_awards)
sorted_awards

[['best', 'supporting', 'actress', 'in', 'a', 'motion', 'picture'],
 ['best', 'motion', 'picture', 'drama'],
 ['best', 'actor', 'in', 'a', 'motion', 'picture,', 'comedy', 'or', 'musical'],
 ['best', 'director', 'motion', 'picture', '#goldenglobe'],
 ['best', 'supporting', 'actor,', 'motion', 'picture'],
 ['best', 'animated', 'feature', 'film'],
 ['best', 'actress', 'in', 'a', 'tv', 'comedy', 'or', 'musical'],
 ['best', 'actor,', 'drama'],
 ['best',
  'supporting',
  'actress',
  'tv',
  'series,',
  'miniseries,',
  'or',
  'tv',
  'movie'],
 ['best', 'original', 'score', '-', 'motion', 'picture'],
 ['best', 'supporting', 'actor', 'in', 'a', 'motion', 'picture'],
 ['best',
  'original',
  'song',
  'for',
  'skyfall,',
  'because',
  'it',
  'is',
  'not',
  'possible'],
 ['best', 'actor', 'in', 'a', 'miniseries/tv', 'movie'],
 ['best', 'actor', 'in', 'a', 'drama'],
 ['best', 'motion', 'picture', 'screenplay'],
 ['best', 'actor', 'tv', 'series', '-', 'comedy', 'or', 'musical'],
 ['best

In [28]:
name = [' '.join(i) for i in sorted_awards]

In [29]:
name

['best supporting actress in a motion picture',
 'best motion picture drama',
 'best actor in a motion picture, comedy or musical',
 'best director motion picture #goldenglobe',
 'best supporting actor, motion picture',
 'best animated feature film',
 'best actress in a tv comedy or musical',
 'best actor, drama',
 'best supporting actress tv series, miniseries, or tv movie',
 'best original score - motion picture',
 'best supporting actor in a motion picture',
 'best original song for skyfall, because it is not possible',
 'best actor in a miniseries/tv movie',
 'best actor in a drama',
 'best motion picture screenplay',
 'best actor tv series - comedy or musical',
 'best film director',
 'best actor in a motion picture drama',
 'best actress in a mini-series/tv movie',
 'best actor in a comedy or musical tv series',
 'best actress in a motion picture drama',
 'best actress, drama,',
 'best director - motion picture',
 'best actor motion picture - drama',
 'best actress in a comedy or

In [41]:
answer = list(answer['award_data'].keys())
answer = [i.lower() for i in answer]

In [43]:
for i in answer:
    if i.lower() in name:
        print(i)

best director - motion picture
best foreign language film
best original score - motion picture
best motion picture - drama
best animated feature film


In [38]:
answer

['hosts', 'award_data']